In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Define our coupled derivatives to integrate

In [4]:
def dydx(x,y):
    
    #set the derivatives
    
    #our equation is d^2y/dx^2 = -y
    
    #so we can write
    #dydx = z
    #dzdx = -y
    
    #we will set y = y[0]
    #we will set z = y[1]
    
    #eclare an array
    y_derivs = np.zeros(2)
    
    #set dydx = z
    y_derivs[0] = y[1]
    
    #set dzdx = z 
    y_derivs[1] = -1*y[0]
    
    #here we have to return an array
    return y_derivs

# Define the CK method

In [25]:
def ck_core(dydx,xi,yi,nv,h):
    
    #declare k? arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    k5 = np.zeros(nv)
    k6 = np.zeros(nv)
    
    #define steps
    x_1 = xi
    
    #x_2 step
    x_2 = xi + (0.2)*h
    
    #x_3 step 
    x_3 = xi + (0.3)*h
    
    #x_4 step 
    x_4 = xi + (0.6)*h
    
    #x_5 step 
    x_5 = xi + (1.0)*h
    
    #x_6 step 
    x_6 = xi + (7./8.)*h
    
    #declare a temp y array
    y_temp = np.zeros(nv)
    
    #get k1 values
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    #get k2 values
    y_temp[:] = yi[:] + 0.2*k1[:]
    y_derivs = dydx(x_2,y_temp)
    k2[:] = h*y_derivs[:]
    
    #get k3 values
    y_temp[:] = yi[:] + (3.0/40.0)*k1[:] + (9.0/40.0)*k2[:]
    y_derivs = dydx(x_3,y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_temp[:] = yi[:] + (0.3)*k1[:]+ (-0.9)*k2[:]+ (6.0/5.0)*k3[:]
    y_derivs = dydx(x_4,y_temp)
    k4[:] = h*y_derivs[:]
    
    #get k5 values
    y_temp[:] = yi[:] + (-11.0/54.0)*k1[:]+ (5.0/2.0)*k2[:]+ (-70.0/27.0)*k3[:] + (35.0/27.0)*k4[:]
    y_derivs = dydx(x_5,y_temp)
    k4[:] = h*y_derivs[:]
    
    #get k6 values
    y_temp[:] = yi[:] + k3[:] + (1631.0/55296.0)*k1[:]+ (175.0/512.0)*k2[:]+ (575.0/13824.0)*k3[:] + (44275.0/110592.0)*k4[:] + (253.0/4096.0)*k5[:]
    y_derivs = dydx(x_6,y_temp)
    k4[:] = h*y_derivs[:]
    
    
    #advance by a step h
    yipo = yi + (37.0/378.0)*k1 + (0.0)*k2 + (250.0/621.0)*k3 +(125.0/594.0)*k4 + (0.0)*k5 + (512.0/1771.0)*k6 
    yipo_star = yi + (2825.0/27648.0)*k1 + (0.0)*k2 + (18575.0/48384.0)*k3 +(13525.0/55296.0)*k4 + (227.0/14336.0)*k5 + (1.0/4.0)*k6
    
    return yipo, yipo_star
    

# Define an adaptive step size driver for RK4

In [32]:
def ck_ad(dydx,x_i,y_i,nv,h,tol):
    
    #define safety scale
    SAFETY   = 0.9
    H_NEW_FAC= 2.0
    
    #set a maximum number of iterations
    imax = 1000
    
    #set an iteration variable
    i = 0
    
    #create an error
    Delta = np.full(nv,2*tol)
    
    #remember the step
    h_step = h
    
    #adjust the stuep
    while(Delta.max()/tol > 1.0):
        
        #would you mind actually commenting and explaining this step here:
        
        #estimate our error by taking one step of size h_step
        y, y_star = ck_core(dydx, x_i, y_i, nv, h_step)
        
        #y_1 = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        #y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_1,nv,0.5*h_step)
        
        #compputer an error
        Delta = np.fabs(y_star-y)
        
        #if the error is too large, take a smaller step
        if(Delta.max()/tol > 1.0):
            
            #our error is too large, decrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        #check iteration
        if(i>=imax):
            print("Too many iteration in ck_ad()")
            raise StopIteration("Ending after i = ",i)
            
        #iteration
        i+=1
        
    #next time, try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9),h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we actually took
    return y,h_new, h_step

# Define a wrapper RK4

In [33]:
def ck_wrapper(dfdx,a,b,y_a,tol):
    
    #dfdx is teh derivative wrt x
    # a is the lower bound
    #b is the upper bound
    #y_a are the boundary conditions
    #tol is the tolerance for integrating y
    
    #define our starting step
    xi = a
    yi = y_a.copy()
    
    #an intitial step size == make very small!
    h = 1.0e-4 * (b-a)
    
    #set a maximum number of iterations
    imax = 1000
    
    #set an iteration ariable
    i=0
    
    #set teh number of coupled odes to the size of y_a
    nv = len(y_a)
    
    #set teh initial conditions
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    #set a flag
    flag = 1
    
    #loop until we reach the right side
    while(flag):
        
        #calculate y_i+1
        yi_new, h_new, h_step = ck_ad(dydx,xi,yi,nv,h,tol)
        
        #update the step
        h = h_new
        
        #prevent an overshoot
        if(xi+h_step>b):
            
            #take a smaller step
            h = b-xi
            
            #recalculatine y_i+1
            yi_new, h_new, h_step = ck_ad(dydx,xi,yi,nv,h,tol)
            
            #break
            flag = 0
            
        #update values
        xi += h_step
        yi[:] =yi_new[:]
        
        #add teh step to the arrays
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y
        y = y_new
        
        #prevent too many iterations
        if(i>imax):
            
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number = ",i)
            
        #iterate
        i+= 1
        
        #output some information
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i,xi, h_step, b)
        print(s)
        
        #break if new xi is == b
        if(xi==b):
            flag = 0
            
    #return the answer
    return x,y

# Perform the integration

In [34]:
a = 0.0
b = 2.0 * np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

#performt he integration
x,y = ck_wrapper(dydx,a,b,y_0,tolerance)

TypeError: unsupported operand type(s) for -: 'tuple' and 'tuple'

# plot the result

In [11]:
plt.plot(x,y[:,0],'o',label='y(x)')
plt.plot(x,y[:,1],'o',label='dydx(x)')
xx = np.linspace(0,2.0*np.pi,1000)
plt.plot(xx,np.sin(xx),label='sinx')
plt.plot(xx,np.cos(xx),label='cosx')
plt.xlabel('x')
plt.ylabel('y, dy/dx')
plt.legend(frameon=False)

NameError: name 'x' is not defined

# Plot the error
Notice that the errors will actually exceed our "tolerance"

In [ ]:
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0]-sine)
dydx_error = (y[:,1]-cosine)

plt.plot(x,y_error, label="y(x) Error")
plt.plot(x, dydx_error, label="dydx(x) Error")
plt.legend(frameon=False)